In [1]:
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat

In [2]:
import networkx as nx

In [3]:
G = nx.DiGraph()

In [4]:
airports = open("airports.dat", "r")
while True:
    line = airports.readline()

    if not line:
        break

    p = line.split(",")

    #wrongly formated lines
    if p[4] == "\\N" or len(p[4])!=5: continue;

    G.add_node(p[4].strip('\"'), pos=(p[6],p[7]))

airports.close()

In [5]:
print(len(G.nodes))
print(len(G.edges))

6054
0


In [6]:
routes = open("routes.dat", "r")

while True:
    line = routes.readline()

    if not line:
        break

    p = line.split(",")
    G.add_edge(p[2], p[4])

routes.close()

In [7]:
print(len(G.nodes))
print(len(G.edges))

6230
37595


In [8]:
H = G.copy()
for node in H.nodes(data=True) :
  if node[1] == {}:
    G.remove_node(node[0])

In [9]:
print(len(G.nodes))
print(len(G.edges))

6054
36768


In [10]:
G.is_directed()

True

In [11]:
nodes = sorted(G.degree, key=lambda x: x[1])
print(nodes)

[('HFN', 0), ('HZK', 0), ('PFJ', 0), ('SIJ', 0), ('VEY', 0), ('YAZ', 0), ('YBB', 0), ('YCH', 0), ('YCT', 0), ('YCW', 0), ('YDA', 0), ('YDB', 0), ('YDL', 0), ('YDN', 0), ('YEN', 0), ('YET', 0), ('YEU', 0), ('YFE', 0), ('YFR', 0), ('YGQ', 0), ('YHB', 0), ('YIB', 0), ('YJN', 0), ('YKY', 0), ('YKZ', 0), ('YLD', 0), ('YLJ', 0), ('YLT', 0), ('YMA', 0), ('YMJ', 0), ('YMW', 0), ('YMX', 0), ('YND', 0), ('YNM', 0), ('YOC', 0), ('YOD', 0), ('YPA', 0), ('YPE', 0), ('YPG', 0), ('YPQ', 0), ('YQA', 0), ('YQH', 0), ('YQV', 0), ('YQW', 0), ('YRI', 0), ('YRJ', 0), ('YRM', 0), ('YSC', 0), ('YSR', 0), ('YSU', 0), ('YTR', 0), ('YVC', 0), ('YVG', 0), ('YVT', 0), ('YVV', 0), ('YWA', 0), ('YWY', 0), ('YXD', 0), ('YXR', 0), ('YYL', 0), ('YYN', 0), ('YYW', 0), ('YZD', 0), ('YZE', 0), ('YZH', 0), ('YZU', 0), ('YZW', 0), ('YZX', 0), ('ZFA', 0), ('QLD', 0), ('BUJ', 0), ('VVZ', 0), ('GJL', 0), ('TIN', 0), ('TAF', 0), ('BFW', 0), ('LOO', 0), ('TMX', 0), ('BYK', 0), ('DJO', 0), ('HGO', 0), ('MJC', 0), ('SPY', 0), ('A

In [12]:
for node in nodes :
  if node[1] == 0:
    G.remove_node(node[0])

In [13]:
nodes = sorted(G.degree, key=lambda x: x[1])
print(nodes)

[('FMI', 1), ('TTA', 1), ('KZI', 1), ('VDA', 1), ('TUA', 1), ('LUR', 1), ('TLJ', 1), ('PTU', 1), ('MSW', 1), ('UII', 1), ('LJA', 1), ('JQE', 1), ('IUE', 1), ('PTJ', 1), ('DLZ', 1), ('IGG', 1), ('PTH', 1), ('CHU', 1), ('KUK', 1), ('AGN', 1), ('ELV', 1), ('KPV', 1), ('PIP', 1), ('KYK', 1), ('SLQ', 1), ('MLY', 1), ('CMP', 1), ('BVS', 1), ('STZ', 1), ('SXX', 1), ('SXK', 1), ('QFG', 1), ('THU', 2), ('AEY', 2), ('EGS', 2), ('IFJ', 2), ('YBR', 2), ('YFS', 2), ('YGK', 2), ('YJT', 2), ('YLL', 2), ('YPL', 2), ('YPR', 2), ('YQL', 2), ('YUB', 2), ('YVM', 2), ('YXH', 2), ('YYF', 2), ('YZP', 2), ('YZR', 2), ('YZT', 2), ('ZFM', 2), ('MZW', 2), ('TEE', 2), ('CFK', 2), ('TGR', 2), ('TML', 2), ('NYI', 2), ('TKD', 2), ('IBA', 2), ('ILR', 2), ('JOS', 2), ('KAD', 2), ('SKO', 2), ('YOL', 2), ('ANR', 2), ('OST', 2), ('MHG', 2), ('XFW', 2), ('AGB', 2), ('URE', 2), ('TAY', 2), ('IVL', 2), ('JOE', 2), ('KAJ', 2), ('KAO', 2), ('KTT', 2), ('KUO', 2), ('RVN', 2), ('SVL', 2), ('GLO', 2), ('ILY', 2), ('DND', 2), ('T

In [14]:
print(len(G.nodes))
print(len(G.edges))

3243
36768


In [15]:
print(G.degree("LIS"))

202


In [16]:
# Graph density
nx.density(G)

0.0034971160776601736

In [17]:
# Graph diameter
nx.diameter(G)

NetworkXError: Found infinite path length because the digraph is not strongly connected

In [20]:
# compute graph degree centrality
dc = nx.degree_centrality(G)
print(max(dc))
print(min(dc))

ZYL
AAE


In [21]:
# compute eigen vector centrality
ec = nx.eigenvector_centrality(G)
print(max(ec))
print(min(ec))

ZYL
AAE


In [22]:
# compute closeness centrality
cc = nx.closeness_centrality(G)
print(max(cc))
print(min(cc))

ZYL
AAE


In [23]:
# compute betweenness centrality
bc = nx.betweenness_centrality(G)
print(max(bc))
print(min(bc))

ZYL
AAE


In [26]:
# compute pagerank
pr = nx.pagerank(G)
print(max(pr))
print(min(pr))

ZYL
AAE
